In [1]:
import yaml
import os
import pandas as pd
from keras.models import model_from_yaml
from LSTM.lstm_sa_test import lstm_predict

Using TensorFlow backend.


In [2]:
basic_path = os.getcwd()
print('loading model......')
with open(basic_path + '/LSTM/lstm.yml', 'r') as f:
    yaml_string = yaml.load(f)
model = model_from_yaml(yaml_string)
print('loading weights......')
model.load_weights(basic_path + '/LSTM/lstm.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

loading model......
Instructions for updating:
Colocations handled automatically by placer.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
loading weights......


In [3]:
def Lstm(strings):
    return lstm_predict(model, strings)

In [4]:
class SentimentAnalysis:
    def __init__(self, articles):
        self.articles = articles
        self.weights = [1.0, 0.6, 0.5, 0.4, 0.3]

    def evaluate(self, article):
        evaluations = {}
        paragraphs = article.split("\n\n\n")
        for i, paragraph in enumerate(paragraphs):
            if i == 0:
                evaluations[paragraph] = self.weights[0]
            else:
                sentences = paragraph.split("\n")
                for j, sentence in enumerate(sentences):
                    if i == 1:
                        if j == 0:
                            evaluations[sentence] = self.weights[1]
                        else:
                            evaluations[sentence] = self.weights[2]
                    else:
                        if j == 0:
                            evaluations[sentence] = self.weights[3]
                        else:
                            evaluations[sentence] = self.weights[4]
        scores = pd.DataFrame(lstm_predict(model, list(evaluations.keys())))
        print(scores)
        scores["weights"] = scores.iloc[:, 3].apply(lambda x: evaluations[x])
        scores["Neural"] = scores.iloc[:, 0].astype('float') * scores["weights"]
        scores["Positive"] = scores.iloc[:, 1].astype('float') * scores["weights"]
        scores["Negtive"] = scores.iloc[:, 2].astype('float') * scores["weights"]
        return scores[["Neural", "Positive", "Negtive"]].mean(axis=0)

    def score(self, ouput_file):
        w = open(ouput_file, "w")
        for article in self.articles:
            results = self.evaluate(article)
            score = results[results == results.max()]
            w.write(str(score.index[0]) + "\n")
        w.close()

In [5]:
if __name__ == "__main__":
    articles = open(basic_path + "/Data/Test.txt", "r").read().split("\n\n\n\n")
    sentiment = SentimentAnalysis(articles)
    sentiment.score(basic_path + "/Data/Test_classification.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3b/wnl918tx4kj0flqblcc2_7500000gn/T/jieba.cache
Loading model cost 0.861 seconds.
Prefix dict has been built succesfully.


             0           1           2  \
0    0.5759844  0.21202034  0.21199524   
1    0.5628386  0.22241902   0.2147424   
2   0.57604903  0.21196726   0.2119837   
3   0.57494473  0.21284443  0.21221085   
4    0.5747367  0.21290222  0.21236108   
5   0.57555026  0.21225299  0.21219677   
6     0.509064   0.2549494  0.23598662   
7    0.5614334  0.21775395  0.22081266   
8   0.57604825   0.2119691  0.21198271   
9   0.57594174  0.21203372  0.21202454   
10   0.5759329  0.21204714  0.21201998   
11  0.56749964  0.21748008  0.21502027   
12  0.57588446  0.21203122  0.21208435   
13   0.5759903    0.211998  0.21201168   
14   0.5760611  0.21196635  0.21197252   
15   0.5760279  0.21198533  0.21198678   
16  0.21349292    0.574037   0.2124701   
17   0.5758843  0.21201088   0.2121049   

                                                    3  
0                               特朗普想买下格陵兰岛，丹麦回应：开玩笑吧？  
1   【环球网报道 见习记者 张晓雅】美国总统特朗普15日被曝有意购买下世界最大的岛——格陵兰岛后...  
2   路透社当日报道称，丹麦政界人士对于特朗普这一“奢侈”的想法